In [46]:
import numpy as np

import plotly.graph_objects as go

from rlway.pyosrd import OSRD

In [47]:
sim = OSRD(use_case='point_switch', dir='point_switch')

[16:15:26,807] [DEBUG]             [Allowance] computing range n°1
[16:15:26,837] [DEBUG]             [Allowance]   computing section n°1
[16:15:26,838] [DEBUG]             [Allowance]   target time = 471.66945
[16:15:26,839] [DEBUG]             [Allowance]     starting attempt 1
[16:15:26,850] [DEBUG]             [Allowance]     envelope time 809.252
[16:15:26,851] [DEBUG]             [Allowance]     starting attempt 2
[16:15:26,854] [DEBUG]             [Allowance]     envelope time 577.684
[16:15:26,854] [DEBUG]             [Allowance]     starting attempt 3
[16:15:26,856] [DEBUG]             [Allowance]     envelope time 505.361
[16:15:26,857] [DEBUG]             [Allowance]     starting attempt 4
[16:15:26,858] [DEBUG]             [Allowance]     envelope time 475.49
[16:15:26,860] [DEBUG]             [Allowance]     starting attempt 5
[16:15:26,861] [DEBUG]             [Allowance]     envelope time 462.012
[16:15:26,863] [DEBUG]             [Allowance]     starting attempt 6
[16:1

In [48]:
# sim=OSRD('../cases/small_infra/')

In [49]:
def fix_departure_times(self) -> None:
    """Fix departure times (useful if all are set to 0 due to CLI API bug)"""
    for train, _ in enumerate(self.trains):
        for eco_or_base in ['base', 'eco']:
            for record in self.results[train][f"{eco_or_base}_simulations"][0]['head_positions']:
                record['time'] += self.departure_times[train]

fix_departure_times(sim)

In [57]:
from typing import Tuple, List

def path(self, train_id: int, eco_or_base: str = 'eco') -> Tuple[List[float], List[float]]:
    """Path offset of a train as a function of time"""
    simulation = f"{eco_or_base}_simulations"
    records = [
        record
        for record in sim.results[train_id][f"{eco_or_base}_simulations"][0]['head_positions']
        # if record['track_section'] in common_tracks
    ]
    offset = self.results[train_id][f"{eco_or_base}_simulations"][0]['head_positions'][0]['offset']
    offsets = [offset + t['path_offset'] for t in records]
    times = [t['time']/60 for t in records]

    return times, offsets

train = 0
eco_or_base = 'eco'

times, offsets = path(sim, train, eco_or_base)


In [58]:
train2 = 1

common_tracks = [
    t
    for t in sim.train_track_sections(train2)
    if t in sim.train_track_sections(train)
]

In [59]:
if common_tracks:

     first_common_switch = list(sim.points_on_track_sections[common_tracks[0]].keys())[0]

     offset_common_switch_path_1 = [
          point['path_offset']
          for point in sim.points_encountered_by_train(0)
          if point['id'] == first_common_switch
          ][0]
     offset_common_switch_path_2 = [
          point['path_offset']
          for point in sim.points_encountered_by_train(train2)
          if point['id'] == first_common_switch
     ][0]
     offsets_2 = [
          record['path_offset'] - offset_common_switch_path_2 + offset_common_switch_path_1
          if record['track_section'] in common_tracks
          else np.nan
          for record in sim.results[train2][f"{eco_or_base}_simulations"][0]['head_positions']
     ]
     times_2 = [
          record['time']/60
          if record['track_section'] in common_tracks
          else np.nan
          for record in sim.results[train2][f"{eco_or_base}_simulations"][0]['head_positions']
     ]

In [60]:
points = sim.points_encountered_by_train(train, ['station', 'switch', 'detector', 'signal'])

In [61]:
fig = go.Figure(
    data = [
        go.Scatter(x=times, y=offsets, name='train0'),
        go.Scatter(x=times_2, y=offsets_2, mode='lines', name='train1'),
    ],
    layout = {
        "title": 'train 0 (eco)',
        "template": "simple_white",
    },
)

for point in points:
    fig.add_hline(
        y=point['path_offset'],
        line_width=.5,
        line_dash="dash",
        line_color="black"
    )

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [point['path_offset'] for point in points],
        ticktext = [point['id'] for point in points]
    )
)


## Problèmes

### Offset

Les path_offset es autres trains doivent être positionnés **dans le repère du path du train affiché**. Par exemple ici, pb de sens.
  --> Méthode de conversion `offset_in_path_of_train(track: str, position: float) -> Union[float, None]` (ou sur une liste pour ne pas tout recalculer à chaque fois ?)

In [65]:
from typing import Union

def offset_in_path_of_train(self, track: str, position: float, ) -> float:
    """Calculates the offset of a point on a track relative to the path of a given train"""
    if track not in self.train_track_sections:
        return np.nan
    ...
    
sim.train_track_sections(0)
    

{'T0': 'START_TO_STOP', 'T1': 'START_TO_STOP'}